In [ ]:
#For Local Machine
import findspark
findspark.init()
findspark.find()

In [ ]:
#Using SparkContext, SparkConf
from pyspark import SparkConf,SparkContext
conf=SparkConf().setMaster('local').setAppName('Testing')
sc=SparkContext(conf=conf)

In [ ]:
#config('spark.ui.port','0')

In [5]:
#Using SparkSession
from pyspark.sql import SparkSession

spark=SparkSession. \
builder. \
appName('Testing'). \
master('local'). \
getOrCreate()

In [45]:
spark

In [7]:
sc=spark.sparkContext

In [8]:
sc

<SparkContext master=local appName=Testing>

In [9]:
import os

In [10]:
os.getcwd()

'/home/vagrant/SparkPractices'

In [11]:
#Reading from HDFS
orderItems=sc.textFile('/public/retail_db/order_items')

In [37]:
#Reading from local FS - NOTE: Only in local mode.
blog=sc.textFile('file:///home/vagrant/SparkPractices/Share/blogtexts')

In [40]:
type(blog)

pyspark.rdd.RDD

In [41]:
orderItems

/public/retail_db/order_items MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

## RDD APIs

In [42]:
#Printing the first element of RDD
print(orderItems.first())
print(blog.first())

1,1,957,1,299.98,299.98
Think of it for a moment – 1 Qunitillion = 1 Million Billion! Can you imagine how many drives / CDs / Blue-ray DVDs would be required to store them? It is difficult to imagine this scale of data generation even as a data science professional. While this pace of data generation is very exciting,  it has created entirely new set of challenges and has forced us to find new ways to handle Big Huge data effectively.


In [43]:
#Counting number of items in RDD
print(orderItems.count())
print(blog.count())

172198
484


In [23]:
#Looping using Take function
for i in orderItems.take(10):
    print(i)

1,1,957,1,299.98,299.98
2,2,1073,1,199.99,199.99
3,2,502,5,250.0,50.0
4,2,403,1,129.99,129.99
5,4,897,2,49.98,24.99
6,4,365,5,299.95,59.99
7,4,502,3,150.0,50.0
8,4,1014,4,199.92,49.98
9,5,957,1,299.98,299.98
10,5,365,5,299.95,59.99
